In [1]:
import json
import logging
import re
import warnings
from pathlib import Path
from pprint import pprint
from typing import Annotated, Any, Generator, Literal, Type, TypeVar

# Standard imports
import numpy as np
import numpy.typing as npt
import pandas as pd
import polars as pl

# Visualization
# import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(n=200)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)


def create_path(path: str | Path) -> None:
    """
    Create parent directories for the given path if they don't exist.

    Parameters
    ----------
    path : str | Path
        The file path for which to create parent directories.

    """
    Path(path).parent.mkdir(parents=True, exist_ok=True)


def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.

    Params:
    -------
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.

    Returns:
    --------
    None
    """
    import os
    import sys

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)

In [3]:
go_up_from_current_directory(go_up=1)

/Users/neidu/Desktop/Projects/Personal/batch-process


In [4]:
from config import app_config

In [6]:
console.print(app_config)

AppConfig(
    data=Data(
        data_path='data/train.parquet',
        batch_data=BatchData(
            is_remote=True,
            remote_data_id='1lSKBgYQ6bgV4ARRpeRfPMpLgf1GgyWY8',
            download_path='data/downloaded_data.parquet',
            batch_mode=False,
            batch_size=100
        ),
        num_vars=['age', 'pclass', 'sibsp', 'parch', 'fare', 'survived'],
        cat_vars=['sex', 'embarked']
    ),
    db=DB(db_path='sqlite:///results.db', max_connections=5),
    model=Model(
        hyperparams=ModelHyperparams(n_splits=5, n_estimators=100, max_depth=10, random_state=42, test_size=0.2),
        artifacts=Artifacts(model_path='models/model.pkl')
    ),
    celery_config=CeleryConfig(
        broker_url='pyamqp://guest@localhost:5672//',
        result_backend='redis://localhost:6379',
        task_config=TaskConfig(task_serializer='json', result_serializer='json', timezone='UTC', enable_utc=True),
        task_routes={
            'app.tasks.email_tasks.*': QueueConfig(queue='email'),
            'app.tasks.dataprocessing.*': QueueConfig(queue='data'),
            'app.tasks.periodic_tasks.*': QueueConfig(queue='periodic')
        },
        worker_config=WorkerConfig(
            worker_prefetch_multiplier=1,
            task_acks_late=True,
            worker_max_tasks_per_child=1000
        ),
        beat_config=BeatConfig(
            beat_schedule=BeatSchedule(
                cleanup_old_records=TaskAndSchedule(
                    task='app.tasks.periodic_tasks.cleanup_old_records',
                    schedule=3600
                )
            ),
            health_check=TaskAndSchedule(task='app.tasks.periodic_tasks.health_check', schedule=300)
        )
    )
)

In [7]:
from contextlib import contextmanager

session = ...


@contextmanager
def get_db_session() -> Generator["Session", None, None]:
    try:
        yield session
        session.commit()
    except Exception:
        session.rollback()
        raise
    finally:
        session.close()

In [11]:
len("hkavhfgihaggaihakjghdfjkgfazvzafjafhjahfijoafhjkah")

50